In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-employee-access-challenge/sampleSubmission.csv
/kaggle/input/amazon-employee-access-challenge/train.csv
/kaggle/input/amazon-employee-access-challenge/test.csv


In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score
import pickle


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
amazon =  pd.read_csv("/kaggle/input/amazon-employee-access-challenge/train.csv")
test=  pd.read_csv("/kaggle/input/amazon-employee-access-challenge/test.csv")
test.drop("id", axis =1, inplace =True)

In [4]:
X =  amazon.iloc[: , 1:]
y= amazon.iloc[: ,0 ]
X['Belong'] = 0 
test['Belong'] = 1 

In [5]:
X =  pd.concat([X , test])

# Frequency Encoding


In [6]:
for z,col in enumerate(X):
    
    frequency_map = X[col].value_counts(normalize=True)

    X[col+f"_{z}"] = X[col].map(frequency_map)

# One Hot Encoding

In [7]:
test =  X[X['Belong'] ==1].drop(['Belong', 'Belong_9'], axis=1)

In [8]:
X =  X[X['Belong'] ==0 ].drop(['Belong', 'Belong_9'], axis=1)

In [9]:
X= pd.get_dummies(X , columns= ["ROLE_FAMILY"] )

In [10]:
test =  pd.get_dummies(test , columns= ["ROLE_FAMILY"])

In [11]:
col = []
for x in X.columns:
    for z in test.columns:
        if x==z:
            col.append(x)
            break 
               

In [12]:
test = test.loc[: , col]

In [13]:
X= X.loc[: ,col].values

# XGboost

In [14]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import confusion_matrix 
import xgboost as xgb

neg, pos = np.bincount(y)
scale_pos_weight = neg / pos


classifier = xgb.XGBClassifier( eta = 0.04, n_estimators = 200, max_depth= 25,

gamma = 0.5, min_child_weight =1,

subsample = 0.8, colsample_bytree = 0.35 ,

 alpha   =  0.8,
 reg_lambda  = 0.05,
scale_pos_weight=scale_pos_weight


) 


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) 

confusion_matrices = []

for train_index, test_index in skf.split(X, y): 
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]

    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    confusion_matrices.append(cm)

avg_confusion_matrix = np.mean(confusion_matrices, axis=0)


df_avg_confusion_matrix = pd.DataFrame(avg_confusion_matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]) 
print("Average Confusion Matrix:") 
print(df_avg_confusion_matrix)

Average Confusion Matrix:
          Predicted 0  Predicted 1
Actual 0        253.6        125.8
Actual 1        527.4       5647.0


In [15]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score
import xgboost as xgb 
import numpy as np


# Define the model
model = xgb.XGBClassifier()

# Define the hyperparameters to optimize
param_grid = {
    "eta": [0.05] , "n_estimators" : [300] ,
    "max_depth" : [50] ,
 
   "gamma" : [0.5] ,
  "min_child_weight"   : [1],
  
"subsample" :  [0.8] ,
 "colsample_bytree"  :  [0.35] ,

     "alpha" :   [0.8],
     "reg_lambda" :[0.05],
    "scale_pos_weight" : [scale_pos_weight]
}

# Set up the StratifiedKFold object
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the scoring method


# Set up the GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring= "roc_auc", cv=kfold, verbose= 3)

# Fit the GridSearchCV object to the data
grid_search.fit(X, y)

# Get the results of the grid search
print("Best parameters: ", grid_search.best_params_)
print("Best AUC score: ", grid_search.best_score_)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END alpha=0.8, colsample_bytree=0.35, eta=0.05, gamma=0.5, max_depth=50, min_child_weight=1, n_estimators=300, reg_lambda=0.05, scale_pos_weight=0.06144726613112205, subsample=0.8;, score=0.875 total time=  19.7s
[CV 2/5] END alpha=0.8, colsample_bytree=0.35, eta=0.05, gamma=0.5, max_depth=50, min_child_weight=1, n_estimators=300, reg_lambda=0.05, scale_pos_weight=0.06144726613112205, subsample=0.8;, score=0.854 total time=  20.7s
[CV 3/5] END alpha=0.8, colsample_bytree=0.35, eta=0.05, gamma=0.5, max_depth=50, min_child_weight=1, n_estimators=300, reg_lambda=0.05, scale_pos_weight=0.06144726613112205, subsample=0.8;, score=0.869 total time=  20.6s
[CV 4/5] END alpha=0.8, colsample_bytree=0.35, eta=0.05, gamma=0.5, max_depth=50, min_child_weight=1, n_estimators=300, reg_lambda=0.05, scale_pos_weight=0.06144726613112205, subsample=0.8;, score=0.879 total time=  19.3s
[CV 5/5] END alpha=0.8, colsample_bytree=0.35, eta=0.

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
import xgboost as xgb


neg, pos = np.bincount(y)
scale_pos_weight = neg / pos


classifier = xgb.XGBClassifier( eta = 0.04, n_estimators = 200,
    max_depth= 25,
 
   gamma = 0.5,
  min_child_weight   =1,
  
subsample   = 0.8,
 colsample_bytree   = 0.35 ,

     alpha   =  0.8,
     reg_lambda  = 0.05,
    scale_pos_weight=scale_pos_weight
                               
    
    
) 
classifier.fit(X,y)

XGBClassifier(alpha=0.8, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.35, early_stopping_rounds=None,
              enable_categorical=False, eta=0.04, eval_metric=None,
              feature_types=None, gamma=0.5, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=25,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [17]:
prediction = classifier.predict_proba(test.values)[:, 1]

In [18]:
submission = pd.read_csv("/kaggle/input/amazon-employee-access-challenge/sampleSubmission.csv")

In [19]:
submission['Action']= prediction 

In [20]:
submission.to_csv("submission.csv", index = False)